<a href="https://colab.research.google.com/github/RafaelNovais/MasterAI/blob/master/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Rafael Novais de Melo -
23113607**

### Problem definition

A company ABC have 1238 employees. ABC wants to buy a new insurance policy for its employees. Attach is a dataset (train-2.csv Download train-2.csv) about some of their previous employees where the company paid medical costs billed by a health insurance company XYZ. The total cost was 16410282.7 euros.

ABC recently recruited about 100 additional employees. XYZ gave them a high price (i.e. 17755825 euros) for the same policy considering there will be additional 100 employees.

You are working in ABC as a data analyst/research engineer. Your boss asked you to rapidly use the existing data that you have to see whether the given price is reasonable or not. Can you accurately predict the costs for the new employees based on their given information? Report the total price (17755825 euros) that is the previous total price (16410282.7) plus the new predicted price for the new employees with specific features given in the testing set (test-3.csv Download test-3.csv).

Do the following:

First, report what type of algorithm you need to use and why? [1 Mark]
Use all the techniques of pre-processing, cleaning, and normalisation to prepare your data. Report each with justification why you want to use. [3 Marks]
Analyse the data by visualising each feature or overall whole dataset with various graphs, bars, etc. to understand the data before applying a model. Report what you learned from visualising the data. Did you find any correlation or discrepancies in the data. [3 Marks]
Use the k-fold cross validation approach to find the optimal model that you can apply on the testing data provided. Report the performance e.g loss and loss curve,  validation accuracy and its curve (if applicable) and show when and why you stopped e.g., did you reached convergence or not. [2.75 Marks]
Report the final total price that you got. [0.25 Marks]
This time you need to apply and present the results in the best way possible to your boss. Considering you have learned various ways.

Upload your pdf of not more than 4 pages in the area designated for pdf upload.

In addition, upload the code in the designated area for code upload. Plagiarism will be strictly penalised.

In [49]:
#Read and import file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from sklearn.base import BaseEstimator
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
import seaborn as sns
from scipy.spatial import ConvexHull
import requests
from io import StringIO
from sklearn.model_selection import train_test_split


df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train-2.csv')
#url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

#Validation Data Prepared
#df = upload_file()
cleaned_df = clean_df(df)
#test_df = upload_file()
#cleaned_test_df = clean_df(test_df)

#Select stratify data
x_data, y_data = select_data(cleaned_df)


#Slipt the data in Train and Test
x_train, x_test, y_train, y_test = split_data(x_data, y_data)



#Function
#print('Convex Hull')
#convex_hull(cleaned_column)
#print('Isolation Forest')
#isolation_forest(cleaned_column)
#print('Bounding Box')
#bounding_box(cleaned_column)
#print('Outliers')
#outliers(cleaned_column)




Columns in the DataFrame:
age
sex
bmi
children
smoker
region
charges
Enter the name of a column: charges


ValueError: ignored

In [50]:
print(y_data.value_counts())


1639.56310     2
8219.20390     1
13393.75600    1
6117.49450     1
39597.40720    1
              ..
19594.80965    1
1621.34020     1
4074.45370     1
6652.52880     1
12224.35085    1
Name: charges, Length: 1237, dtype: int64


In [51]:
#Upload File

def upload_file(): #Upload/Read csv file
    file_path = input("Please enter the file path or URL to upload: ")
    df = pd.read_csv(file_path)
    return df


In [14]:
#Clean the dataframe
def clean_df(data):
  cleaned_data = data.dropna()
  return cleaned_data


In [47]:
#Select X column and Y column
def select_data(data):
    print("Columns in the DataFrame:")  # Show all column names to select 2
    for column in data.columns:
        print(column)

    column_name = input("Enter the name of a column: ")
    if column_name not in data.columns:
        raise ValueError("Invalid column name. Please enter a valid column name.")

    x_data = data.drop(columns=column_name)  # Drop columns specified in column_name from x_data
    y_data = data[column_name]

    return x_data, y_data


In [48]:
# Split Data into Train/Test
def split_data(x_data, y_data):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=10, stratify=y_data)

    return x_train, x_test, y_train, y_test




In [6]:
#ConvexHull
def convex_hull(data):
    xy = data.values
    hull = ConvexHull(xy) # Convex Hull is a function where the next convex angle is found.
    periphery_indices = hull.vertices #Set the index os all periphery in a variable
    periphery_indices = periphery_indices.astype(int)
    periphery_points = data.iloc[periphery_indices] #Set the all periphery points in the graphic in a variable
    plt.scatter(data.iloc[:, 0], data.iloc[:, 1], s=50, c='b', label='Main') #Plot the all the points in the graphic
    plt.scatter(periphery_points.iloc[:, 0], periphery_points.iloc[:, 1], s=50, c='r', label='Periphery') # Paint the periphery in red and save in graphic
    plt.plot(data.iloc[periphery_indices, 0], data.iloc[periphery_indices, 1], 'k--', lw=2)#Conect all the periphery point with line and save in graphic
    plt.legend()# Display the graphic
    plt.title('Convex Hull')
    plt.xlabel(data.columns[0])
    plt.ylabel(data.columns[1])
    plt.show()# Display the graphic



Convex Hull:
Is good to see the data distribution and the detected outliers.
But is not good with Mult Dimension

In [7]:
#IsolationForest
def isolation_forest(data):

  model = IsolationForest(contamination=0.05)  # Adjust contamination based on your dataset
  model.fit(data) # Fit the model to the data
  outliers = model.predict(data) # Predict anomalies (outliers)
  anomaly_points = data[outliers == -1] # Extract the anomaly points
  plt.scatter(data.iloc[:, 0], data.iloc[:, 1], s=50, c='b', label='Normal Data')
  plt.scatter(anomaly_points.iloc[:, 0], anomaly_points.iloc[:, 1], s=50, c='r', label='Anomaly Points')
  plt.legend()
  plt.xlabel(data.columns[0])
  plt.ylabel(data.columns[1])
  plt.title('Isolation Forest')
  plt.show()

Isolation Forest: s a fast and efficient algorithm for identifying outliers. It's especially useful for large datasets / Mult Dimension. The performace is not good

In [8]:
#Outliers
def outliers(data):
  plt.figure(figsize=(20, 10))
  sns.boxplot(x=data[data.columns[0]], y=data[data.columns[1]], color='lightblue', width=0.6)
  sns.stripplot(x=data[data.columns[0]], y=data[data.columns[1]], color='red', size=4, jitter=0.2)
  plt.xlabel(data.columns[0])
  plt.ylabel(data.columns[1])
  plt.title(f"Outliers")
  plt.show()

Outliers: is good to analise the extremes for each feature and the performace is good to robust data and identify if have any anomaly, but could not be so efective and show some false positives

Bounding Box using ChatGPT
>
Rafael : can you do an exemple of bounding box with python.
>
ChatGPT: Generated
```
# Extract the feature columns
# Calculate the minimum and maximum values for each feature
# Define the bounding box
```
Rafael: how can i do a plot with the anwser in Bounding Box:
```
# Filter points within the bounding box
# Create a scatter plot for all data points
# Create a rectangle for the bounding box
# Plot the
# Highlight points within the bounding box
```
Rafael: ValueError: Can only compare identically-labeled Series objects
---> 19     (X["temp_max"] >= bounding_box["min_temp_max
>
ChatGPT: Fixed the Code, and return the atual code
```
This corrected code should allow you to create a plot with the bounding box and points within the box for your dataset. Make sure to replace "dataset.csv" with the path to your CSV file and adjust the column names as needed.
```














In [9]:
# Bounding Box from ChatGPT
def bounding_box(data):
    # Extract the feature columns
    X = data

    # Calculate the minimum and maximum values for each feature
    min_vals = X.min()
    max_vals = X.max()

    # Define the bounding box using column names
    bounding_box = {
        f'min_{X.columns[0]}': min_vals[0],
        f'max_{X.columns[0]}': max_vals[0],
        f'min_{X.columns[1]}': min_vals[1],
        f'max_{X.columns[1]}': max_vals[1]
    }

    # Filter points within the bounding box
    points_within_box = X[
        (X[X.columns[0]] >= bounding_box[f'min_{X.columns[0]}']) &
        (X[X.columns[0]] <= bounding_box[f'max_{X.columns[0]}']) &
        (X[X.columns[1]] >= bounding_box[f'min_{X.columns[1]}']) &
        (X[X.columns[1]] <= bounding_box[f'max_{X.columns[1]}'])
    ]

    # Create a scatter plot for all data points
    plt.scatter(X[X.columns[0]], X[X.columns[1]], c='b', label='Data Points')

    # Create a rectangle for the bounding box
    rectangle = plt.Rectangle(
        (bounding_box[f'min_{X.columns[0]}'], bounding_box[f'min_{X.columns[1]}']),
        bounding_box[f'max_{X.columns[0]}'] - bounding_box[f'min_{X.columns[0]}'],
        bounding_box[f'max_{X.columns[1]}'] - bounding_box[f'min_{X.columns[1]}'],
        color='r',
        fill=False,
        label='Bounding Box'
    )

    # Plot the rectangle
    plt.gca().add_patch(rectangle)

    # Highlight points within the bounding box
    plt.scatter(points_within_box[X.columns[0]], points_within_box[X.columns[1]], c='g', label='Points in Bounding Box')

    plt.xlabel(X.columns[0])
    plt.ylabel(X.columns[1])
    plt.legend()
    plt.title(f"Bounding Box and Data Points for {X.columns[0]} and {X.columns[1]}")
    plt.show()



Bounding Box: is good to data exploration to find outliers in the data range but sensitive to the range of data values and may not capture outliers

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Periphery of a dataset is important for assessing a model's generalization, robustness, and its ability to handle unseen or extreme data points, including predictive modeling, anomaly detection, and fairness in machine learning.